In [0]:
!cp ./drive/My\ Drive/Colab\ Notebooks/class-MachineLearning/11/movie_review.zip ./
!unzip ./movie_review.zip
!rm -rf ./movie_review/Icon ./movie_review/pos/Icon ./movie_review/neg/Icon

## 1. Load data from files

In [0]:
from sklearn.datasets import load_files

DATA_PATH = './movie_review'

data = load_files(DATA_PATH)
X_raw, Y_raw = data.data, data.target

## 2. Data preprocessing

### 2-1. Text preprocessing

In [0]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

docs = []
stemmer = WordNetLemmatizer()

# text preprocssing
for doc_raw in X_raw:
  # byte to string
  doc = doc_raw.decode('utf-8')
  
  # remove all the special characters
  doc = re.sub(r'\W', ' ', doc)

  # remove all single characters
  doc = re.sub(r'\s+[a-zA-Z]\s+', ' ', doc)
  doc = re.sub(r'\^[a-zA-Z]\s+', ' ', doc)

  # substitute multiple spaces with single space
  doc = re.sub(r'\s+', ' ', doc, flags=re.I)

  # convert to lowercase
  doc = doc.lower()

  # lemmatization
  word_list = doc.split()
  lemma_word_list = []
  for word, tag in pos_tag(word_list):
    main_tag = tag[0].lower()
    lemma_word = ''

    if main_tag in ['a', 'r', 'n', 'v']:
      lemma_word = stemmer.lemmatize(word, main_tag)
    else:
      lemma_word = word

    lemma_word_list.append(lemma_word)

  doc = ' '.join(lemma_word_list)
  docs.append(doc)

### 2-2. Documents to vectors

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

# documents to tfidf vector
vectorizer = CountVectorizer(max_features=1600, min_df=0.01, max_df=0.7, stop_words=stopwords.words('english'))
X_data = vectorizer.fit_transform(docs).toarray()
tfidf_transformer = TfidfTransformer(sublinear_tf=False)
X_data = tfidf_transformer.fit_transform(X_data).toarray()

Y_data = Y_raw

X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.3, shuffle=False)

### 2-3. Move data to GPU

In [0]:
import numpy as np
import torch

# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train = torch.from_numpy(X_train).to(device)
Y_train = torch.from_numpy(Y_train).to(device)

X_test = torch.from_numpy(X_test).to(device)
Y_test = torch.from_numpy(Y_test).to(device)

## 3. Define model and functions for learning neural network

In [0]:
import math

def initialize_weights(fan_in, fan_out):
  # xavier initialization
  return math.sqrt(1 / fan_in) * torch.randn(fan_out, fan_in)

def activation(z):
  return 1 / (1 + torch.exp(-z))

def objective(Y_pred, Y, lam=0, params=None):
  epsilon = 1e-8
  data_fidelity_term = (-1 / Y.shape[0]) * torch.sum(
      Y * torch.log(Y_pred + epsilon) + (1 - Y) * torch.log(1 - Y_pred + epsilon)
  )

  decay_term = 0
  if lam is not 0:
    num_of_params = 0   # n
    sum_of_params = 0   # sigma theta^2
    for param in params.values():
      num_of_params += param.shape[0] * param.shape[1]
      sum_of_params += torch.sum(param ** 2)

    decay_term = (lam / (2 * num_of_params)) * sum_of_params

  return data_fidelity_term + decay_term

def accuracy(Y_pred, Y):
  answer = (Y_pred >= 0.5).float()
  return torch.mean((answer == Y).float())


class FNN:
  def __init__(self, layer_size_list):
    self.num_of_layer = len(layer_size_list) - 1
    self.params = dict()

    # initialize params w and b (w is weights and b is biases)
    for i in range(len(layer_size_list) - 1):
      fan_in = layer_size_list[i]
      fan_out = layer_size_list[i + 1]

      self.params['w' + str(i + 1)] = initialize_weights(fan_in, fan_out).double().to(device)
      self.params['b' + str(i + 1)] = torch.zeros(1, fan_out).double().to(device)

  def forward(self, X):
    forward_results = dict()
    forward_results['a0'] = X

    # z is fully connected layer's result and
    # a is activation function's result
    for i in range(self.num_of_layer):
      prev_a = forward_results['a' + str(i)]
      curr_w = self.params['w' + str(i + 1)]
      curr_b = self.params['b' + str(i + 1)]

      forward_results['z' + str(i + 1)] = torch.matmul(prev_a, curr_w.T) + curr_b
      forward_results['a' + str(i + 1)] = activation(forward_results['z' + str(i + 1)])

    forward_results['Y_pred'] = forward_results['a' + str(self.num_of_layer)]

    return forward_results

  def backward(self, X, Y, forward_results, lam=0):
    grads = dict()

    dz_last = (forward_results['a' + str(self.num_of_layer)] - Y) / X.shape[0]

    # reverse order because of `back`ward propagation
    for i in range(self.num_of_layer)[::-1]:
      curr_a = forward_results['a' + str(i + 1)]
      next_a = forward_results['a' + str(i)]

      if i == (self.num_of_layer - 1):  # if last layer
        grads['z' + str(i + 1)] = (curr_a - Y) / X.shape[0]
      else:
        prev_dz = grads['z' + str(i + 2)]
        prev_w = self.params['w' + str(i + 2)]

        grads['a' + str(i + 1)] = torch.matmul(prev_dz, prev_w)
        grads['z' + str(i + 1)] = curr_a * (1 - curr_a) * grads['a' + str(i + 1)]

      grads['w' + str(i + 1)] = torch.matmul(grads['z' + str(i + 1)].T, next_a)
      grads['b' + str(i + 1)] = torch.sum(grads['z' + str(i + 1)], axis=0).reshape(1, -1)

    # remain only the gradients of w and b
    for key in list(grads.keys()):
      if key.startswith('z') or key.startswith('a'):
        del grads[key]

    # apply gradients of decay term
    if lam is not 0:
      n = 0
      for param in self.params.values():
        n += param.shape[0] * param.shape[1]

      for key in grads.keys():
        grads[key] += (lam / n) * self.params[key]

    return grads